In [30]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re

Fil = pd.read_pickle('../data/raw/Fil.pkl')

In [40]:
with open('../data/processed/65.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [57]:
len(text)
beginning = text[0:1000]

In [58]:
from danlp.models import load_bert_base_model
model = load_bert_base_model()
vecs_embedding, sentence_embedding, tokenized_text =model.embed_text(beginning)

Some weights of the model checkpoint at C:\Users\rune7\.danlp\bert.botxo.pytorch were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
tokenized_text

['folketingets',
 'lov',
 '##sekret',
 '##ariat',
 'erhvervs',
 '-',
 'og',
 'vækst',
 '##ministeren',
 '15',
 '.',
 'januar',
 '2014',
 'vedlagt',
 'fremsender',
 'jeg',
 'min',
 'besvarelse',
 'af',
 'spørgsm',
 '##al',
 's',
 '##73',
 '##9',
 'stillet',
 'af',
 'jakob',
 'engel',
 '-',
 'schmidt',
 '(',
 'v',
 ')',
 'den',
 '7',
 '.',
 'januar',
 '2014',
 '.',
 'med',
 'venlig',
 'hilsen',
 'henrik',
 'sas',
 '##s',
 'larsen',
 'erhvervs',
 '-',
 'og',
 'vækst',
 '##ministeriet',
 'slots',
 '##holms',
 '##gade',
 '10',
 '-',
 '12',
 '12',
 '##16',
 'københavn',
 'k',
 'tlf',
 '.',
 '33',
 '92',
 '33',
 '50',
 'fax',
 '33',
 '12',
 '37',
 '78',
 'cvr',
 '-',
 'nr',
 '10',
 '09',
 '24',
 '85',
 'ev',
 '##m',
 '[UNK]',
 'ev',
 '##m',
 '.',
 'dk',
 'www',
 '.',
 'ev',
 '##m',
 '.',
 'dk',
 '2013',
 '-',
 '14',
 's',
 '73',
 '##9',
 'endeligt',
 'svar',
 'offentligt',
 'erhvervs',
 '-',
 'og',
 'vækst',
 '##ministeren',
 '15',
 '.',
 'januar',
 '2014',
 'besvarelse',
 'af',
 'spørgsm',
 

In [42]:
print(text)

Folketingets Lovsekretariat
ERHVERVS- OG 
VÆKSTMINISTEREN
15. januar 2014
Vedlagt fremsender jeg min besvarelse af spørgsmål S739 stillet af Jakob 
Engel-Schmidt (V)den 7. januar 2014.
Med venlig hilsen
Henrik Sass Larsen
ERHVERVS- OG 
VÆKSTMINISTERIET
Slotsholmsgade 10-12
1216 København K
Tlf.
33 92 33 50
Fax
33 12 37 78
CVR-nr
10 09 24 85
evm@evm.dk
www.evm.dk
2013-14
S 739 endeligt svar
Offentligt
ERHVERVS- OG 
VÆKSTMINISTEREN
15. januar 2014
Besvarelse af spørgsmål S739stillet af Jakob Engel-Schmidt (V)den 7. 
januar 2014.
Spørgsmål:
Hvad vil ministeren gøre for at stoppe den negative udvikling i antallet af 
nyetablerede virksomheder, der jf. artiklen ”Nystartede virksomheder 
rammer ti års bund” i Børsen den 6. januar 2014 satte bundrekord i 2013?
Svar:
Under højkonjunkturen i årene op til finanskrisens start steg antallet af ny-
startede virksomheder kraftigt. Den efterfølgende langvarige lavkonjunktur 
har i sagens natur gjort det vanskeligere at starte ny virksomhed.
Regeringe

In [71]:
#open bad files
import pickle
with open('../data/raw/bad_files.pkl', 'rb') as f:
    bad_files = pickle.load(f)

len(bad_files)

19665

In [72]:
bad_files

defaultdict(None,
            {466: 403,
             2456: 403,
             1180: 403,
             1746: 403,
             2957: 403,
             3009: 403,
             2923: 403,
             3168: 403,
             3175: 403,
             7418: requests.exceptions.MissingSchema("Invalid URL '/index.htm': No scheme supplied. Perhaps you meant http:///index.htm?"),
             8012: requests.exceptions.MissingSchema("Invalid URL '/index.htm': No scheme supplied. Perhaps you meant http:///index.htm?"),
             3197: 403,
             3196: 403,
             2873: 403,
             20028: 403,
             20030: 403,
             20029: 403,
             35605: 403,
             462: 500,
             2875: 500,
             37955: 403,
             2853: 500,
             37703: 403,
             4607: 404,
             4653: 404,
             9404: 500,
             4694: 404,
             4664: 404,
             4642: 404,
             9413: 500,
             8792: 404,
  

In [81]:
import requests
import regex as re
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
import PyPDF2
from io import BytesIO
from bs4 import BeautifulSoup


In [85]:
def extract_text_from_html(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup.get_text()

def extract_text_from_pdf(pdf_doc):
    # Create a PdfFileReader object
    pdf_reader = PyPDF2.PdfFileReader(BytesIO(pdf_doc))

    # Initialize an empty string to store the extracted text
    extracted_text = ""

    # Iterate through all the pages in the PDF and extract the text
    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        extracted_text += page.extractText()

    return extracted_text

def process_text(text):
    #removing the sentence "PDF to HTML - Convert PDF files to HTML files"
    text = re.sub(r'PDF to HTML - Convert PDF files to HTML files', '', text)
    #removing newlines when there are more than 2 in a row
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text

output_folder = '../data/processed/'


In [86]:
for key, val in bad_files.items():
    if val == 403:
        try:
            url = Fil.loc[key]['filurl']
            file_path = os.path.join(output_folder, f"{str(key)}.txt")
            response = requests.get(url)
            if response.status_code == 200:
                html_doc = response.text
                raw_text = extract_text_from_html(html_doc)
                text = process_text(raw_text)
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(text)
        except:
            pass
    else:
        pass